In [1]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

In [2]:
# model_name = "/data/opt-350m"
# model_name = "/data/llama-hf/Llama-2-13b-hf"
# model_name = "/data/Mistral-7B-Instruct-v0.2"
# model_name = "mistralai/Mistral-7B-v0.1"
# model_name = "codellama/CodeLlama-13b-Python-hf"
model_name = "bigcode/starcoder"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
device_map = {"": 0}
# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.eval()

model.config.use_cache = False
model.config.pretraining_tp = 1





===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/envs/mytorch/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/envs/mytorch/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/opt/conda/envs/mytorch/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /opt/conda/envs/mytorch did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# if "Llama-2" in model_name or "Mistral" in model_name:
# tokenizer.pad_token = tokenizer.bos_token

bos = tokenizer.bos_token_id
eos = tokenizer.eos_token_id
pad = tokenizer.pad_token_id
print("pre-trained model's BOS EOS and PAD token id:",bos,eos,pad," => It should be 1 2 None")

tokenizer.pad_token_id = 1  # unk. we want this to be different from the eos token
tokenizer.padding_side = "right"

pre-trained model's BOS EOS and PAD token id: 0 0 None  => It should be 1 2 None


In [5]:
# prompt = "beautiful scenery!"
prompt = "ABC is a startup based in New York City and Paris"
# prompt = """from qiskit import QuantumCircuit
 
# qc = QuantumCircuit(5)
# qc.h(0)
# qc.cx(0, range(1, 5))
# qc.measure_all()
# """

prompt =  """import cudaq

kernel = cudaq.make_kernel()
qubits = kernel.qalloc(5)
kernel.h(qubits[0])
kernel.cx(qubits[0],qubits[1:5])
kernel.mz(qubits)
"""

input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to("cuda")

with torch.no_grad():
    loss = model(input_ids = input_ids, labels = input_ids).loss.cpu()
    # perp = torch.exp(loss.item())
perp = torch.exp(torch.tensor(loss.item()))
print(f"{perp.item()}")
# output = model(input_ids)
# output = output.logits.squeeze().to("cpu")
# probs = torch.softmax(output, dim = -1)
# context_size = input_ids.shape[1]
# loss = -torch.mean(torch.log(probs[torch.arange(context_size-1),input_ids[0,1:].cpu()]))

# print(loss_hf, loss)
# perp = torch.exp(-torch.mean(torch.log(probs[torch.arange(context_size-1),input_ids[0,1:].cpu()]))  ).item()


3.9051830768585205


In [23]:
print(torch.exp(torch.tensor(loss.item())))

tensor(2.6205)


In [ ]:
from qiskit import QuantumCircuit
 
qc = QuantumCircuit(5)
qc.h(0)
qc.cx(0, range(1, 5))
qc.measure_all()

import cudaq

kernel = cudaq.make_kernel()
qubits = kernel.qalloc(5)
kernel.h(qubits[0])
kernel.cx(qubits[0],qubits[1:5])
kernel.mz(qubits)
